In [1]:
from __future__ import annotations
from exo import proc, DRAM, Procedure, SchedulingError, Memory
import exo.query_asts as exo_ast
import multiprocessing as mp
import concurrent.futures

import re
import sys
from itertools import chain
from functools import reduce
from typing import Any, Callable, Optional

from psutil import Process, cpu_count
Process().cpu_affinity(list(range(cpu_count())))

from evaluate import PersistentQueue, QUEUE_FILE


sys.setrecursionlimit(10_000)

BRANCHING_THRESHOLD: int = 3
CONTROL_STRUCTURE = (exo_ast.For, exo_ast.If, exo_ast.Proc)
SYMBOL_STRUCTURE = (exo_ast.Read, exo_ast.Reduce, exo_ast.Alloc, exo_ast.Assign)
# TYPED_STRUCTURE = SYMBOL_STRUCTURE + (exo_ast.Const, )
LOOP_SPLIT_SIZES: int = 4

#### Debugging and Profiling tools

In [2]:
from pprint import pprint
import cProfile

def profileit(do_print: bool = False):
    def inner(func):
        def wrapper(*args, **kwargs):
            prof = cProfile.Profile()
            retval = prof.runcall(func, *args, **kwargs)

            if do_print:
                prof.print_stats(sort="cumtime")
            else:
                prof.dump_stats(f"{func.__name__}.profile")

            return retval
        return wrapper
    return inner

### Query AST Classes (copied from the docstring)
---

In this notebook, we work the QAST (as opposed to the LoopIR AST, Memory AST, Parallel AST, C compilation AST, etc.). This is primarily due to ease of programmability, as most features of the code are easily extractable from either a regex or a traversal of the QAST. 

Exo ast nodes are organized into a hierarchy of dataclasses:

- QueryAST
  - `Proc`  ( name : str, args : list[FnArg], assertions : list[Expr],
                        body : list[Stmt],  instruction : Optional[str] )
  - `Stmt`
    - `Assign`    ( name : str,   lhs_type : Type,    idx : list[Expr],
                              rhs  : Expr )
    - `Reduce`    ( name : str,   lhs_type : Type,    idx : list[Expr],
                              rhs  : Expr )
    - `WriteConfig` ( config : Config, field : str,
                              rhs  : Expr )
    - `Pass`      ()
    - `If`        ( cond : Expr,  body : list[Stmt],  orelse : list[Stmt] )
    - `For`       ( name : str,   lo   : Expr,        hi : Expr,
                              body : list[Stmt],  is_par : bool )
    - `Alloc`     ( name : str,   type : Type,        memory : Optional[Memory] )
    - `Call`      ( proc : str,   args : list[Expr] )
    - `WindowStmt`( name : str,   rhs  : Expr )
  - `Expr`
    - `Read`    ( name : str,   idx  : list[Expr],    type : Type )
    - `Const`   ( val  : Any,   type : Type )
    - `USub`    ( arg  : Expr,  type : Type )
    - `BinOp`   ( op   : str,   lhs  : Expr,
                            rhs  : Expr,          type : Type )
    - `BuiltIn` ( func : str,   args : list[Expr],    type : Type )
    - `WindowExpr`( name : str, idx : list[WAccess],  type : Type )
    - `StrideExpr`( name : str, dim : int,            type : Type )
    - `ReadConfig`( config : Config, field : str,     type : Type )
  - `WAccess`
    - `Interval`( lo : Expr, hi : Expr )
    - `Point`( pt : Expr )
  - `FnArg` ( name : str, type : Type, memory : Optional[Memory] )
  - `Type`
    - `R`()
    - `f16`()
    - `f32`()
    - `f64`()
    - `i8`()
    - `i32`()
    - `bool`()
    - `int`()
    - `index`()
    - `size`()
    - `stride`()
    - `tensor`( hi : list[Expr], is_window : bool, type : Type )

## Code

### Infrastructure

#### Tools initialization

In [3]:
def get_exo_proc() -> Procedure:
    @proc
    def generated_operation(
        In: i8[16, 16] @ DRAM,
        Weights: i8[16, 16] @ DRAM,
        Out: i8[16, 16] @ DRAM,
    ):
        for i in seq(0, 16):
            for j in seq(0, 16):
                for k in seq(0, 16):
                    Out[i, j] += In[i, k] * Weights[k, j]

    return generated_operation

PROC_AST: exo_ast.Proc = get_exo_proc().get_ast() # pyright: ignore[reportAssignmentType]
pprint(PROC_AST)

Proc(name='generated_operation',
     args=[FnArg(name='In',
                 type=tensor(hi=[Const(val=16, type=int()),
                                 Const(val=16, type=int())],
                             is_window=False,
                             type=i8()),
                 memory=<class 'exo.memory.DRAM'>),
           FnArg(name='Weights',
                 type=tensor(hi=[Const(val=16, type=int()),
                                 Const(val=16, type=int())],
                             is_window=False,
                             type=i8()),
                 memory=<class 'exo.memory.DRAM'>),
           FnArg(name='Out',
                 type=tensor(hi=[Const(val=16, type=int()),
                                 Const(val=16, type=int())],
                             is_window=False,
                             type=i8()),
                 memory=<class 'exo.memory.DRAM'>)],
     assertions=[],
     body=[For(name='i',
               lo=Const(val=0, type=int()),
       

In [4]:
def extract_c_code(proc: Procedure) -> str:
    """Compile the procedure to C code and remove all header information"""
    return proc.c_code_str().split("#include <stdlib.h>")[-1]

In [5]:
def get_pattern_from_node(ast: exo_ast.QueryAST):
    """Produce a "pattern" necessary to match this node using normal scheduling operations"""
    match ast:
        case exo_ast.For(name=n):
            return f"for {n} in _ : _"
        case _:
            raise NotImplementedError

In [6]:
def parallel_nest_call(lst, mapping, enable: bool, 
    threshold: int = BRANCHING_THRESHOLD,
    d_type: type[list | set] = list,
    reduction = chain
    ):
    """Helper method to enable parallel traversals of the AST"""
    if enable and len(lst) > threshold:
        with mp.Pool() as pool:
            return d_type(reduction(
                *pool.starmap(mapping, [(item, False) for item in lst]
            )))
    return d_type(reduction(*[mapping(item, can_split=enable) for item in lst]))

In [7]:
DTYPE = "i8"
def get_names_and_dimensions(proc: Procedure) -> dict[str, list[int]]:
    """Get all buffers that are defined *somewhere* in the procedure"""
    src_code = str(proc)

    matches = re.findall(f'([a-zA-Z_0-9]*): {DTYPE}\[([^\]]*)\]', src_code)
    dims = {
        name: [ int(d) for d in dims.split(", ") ]
        for name, dims in matches
    }

    return dims
get_names_and_dimensions(get_exo_proc())#, ["Out", "In", "Weights"])

{'In': [16, 16], 'Weights': [16, 16], 'Out': [16, 16]}

#### AST Traversals

In [8]:
def find_all_blocks(mod: exo_ast.QueryAST, can_split: bool = True) -> list[exo_ast.For | exo_ast.If]:
    """Produce a list of the For/If nodes in the ast.Module in depth-wise order."""
    if not isinstance(mod, CONTROL_STRUCTURE):
        return [ ]

    match mod:
        case exo_ast.For(body=b) | exo_ast.If(body=b):
            return [mod] + parallel_nest_call(b, find_all_blocks, enable=can_split)
        case exo_ast.Proc(body=b):
            return parallel_nest_call(b, find_all_blocks, enable=can_split)
        case _:
            raise ValueError

find_all_blocks(PROC_AST)

[For(name='i', lo=Const(val=0, type=int()), hi=Const(val=16, type=int()), body=[For(name='j', lo=Const(val=0, type=int()), hi=Const(val=16, type=int()), body=[For(name='k', lo=Const(val=0, type=int()), hi=Const(val=16, type=int()), body=[Reduce(name='Out', lhs_type=i8(), idx=[Read(name='i', idx=[], type=index()), Read(name='j', idx=[], type=index())], rhs=BinOp(op='*', lhs=Read(name='In', idx=[Read(name='i', idx=[], type=index()), Read(name='k', idx=[], type=index())], type=i8()), rhs=Read(name='Weights', idx=[Read(name='k', idx=[], type=index()), Read(name='j', idx=[], type=index())], type=i8()), type=i8()))], is_par=False)], is_par=False)], is_par=False),
 For(name='j', lo=Const(val=0, type=int()), hi=Const(val=16, type=int()), body=[For(name='k', lo=Const(val=0, type=int()), hi=Const(val=16, type=int()), body=[Reduce(name='Out', lhs_type=i8(), idx=[Read(name='i', idx=[], type=index()), Read(name='j', idx=[], type=index())], rhs=BinOp(op='*', lhs=Read(name='In', idx=[Read(name='i', i

In [9]:
def extract_fors(mod: exo_ast.QueryAST, can_split: bool = True) -> list[exo_ast.For]:
    """Produce a list of the For nodes in the ast.Module in depth-wise order."""
    all_blocks = find_all_blocks(mod, can_split=can_split)
    fors: list[exo_ast.For] = filter(lambda b: isinstance(b, exo_ast.For), all_blocks) # pyright: ignore[reportAssignmentType]
    return list(fors)

extract_fors(PROC_AST)

[For(name='i', lo=Const(val=0, type=int()), hi=Const(val=16, type=int()), body=[For(name='j', lo=Const(val=0, type=int()), hi=Const(val=16, type=int()), body=[For(name='k', lo=Const(val=0, type=int()), hi=Const(val=16, type=int()), body=[Reduce(name='Out', lhs_type=i8(), idx=[Read(name='i', idx=[], type=index()), Read(name='j', idx=[], type=index())], rhs=BinOp(op='*', lhs=Read(name='In', idx=[Read(name='i', idx=[], type=index()), Read(name='k', idx=[], type=index())], type=i8()), rhs=Read(name='Weights', idx=[Read(name='k', idx=[], type=index()), Read(name='j', idx=[], type=index())], type=i8()), type=i8()))], is_par=False)], is_par=False)], is_par=False),
 For(name='j', lo=Const(val=0, type=int()), hi=Const(val=16, type=int()), body=[For(name='k', lo=Const(val=0, type=int()), hi=Const(val=16, type=int()), body=[Reduce(name='Out', lhs_type=i8(), idx=[Read(name='i', idx=[], type=index()), Read(name='j', idx=[], type=index())], rhs=BinOp(op='*', lhs=Read(name='In', idx=[Read(name='i', i

In [10]:
union = lambda iter_of_sets: reduce(lambda a, b: a | b, iter_of_sets, set())

In [11]:
def get_swappable_loops(ast: exo_ast.QueryAST, can_split: bool = True) -> set[tuple[str, str]]:
    """Return a set of all swappable for loops"""
    if not isinstance(ast, CONTROL_STRUCTURE):
        return set([])

    match ast:
        case exo_ast.For(name=name, body=[exo_ast.For(name=inner_name)]):
            return set([(name, inner_name)]) \
                 | get_swappable_loops(ast.body[0], can_split)
        case exo_ast.Proc(body=b) | exo_ast.If(body=b) | exo_ast.For(body=b):
            # unfortunately not as simple as the following:
            # return parallel_nest_call(b, get_swappable_loops, can_split, d_type=set, reduction=union)
            if can_split and len(b) > BRANCHING_THRESHOLD:
                with mp.Pool() as pool:
                    return union(pool.starmap(get_swappable_loops, 
                        [ (node, False) for node in b ] 
                    ))

            return union([
                get_swappable_loops(node, can_split)
                for node in b
            ])
        case _:
            raise ValueError

get_swappable_loops(PROC_AST)

{('i', 'j'), ('j', 'k')}

In [12]:
def find_innermost_loops(mod: exo_ast.QueryAST, can_split: bool = True) -> list[exo_ast.For]:
    """Get all loops with no loops inside them"""
    if not isinstance(mod, CONTROL_STRUCTURE):
        return [ ]

    match mod:
        case exo_ast.Proc(body=b) | exo_ast.If(body=b):
            return parallel_nest_call(b, find_innermost_loops, enable=can_split)
        case exo_ast.For(body=b):
            if any(map(lambda n: isinstance(n, exo_ast.For), b)):
                return parallel_nest_call(b, find_innermost_loops, enable=can_split)
            return [mod]
        case _:
            raise ValueError

find_innermost_loops(PROC_AST)

[For(name='k', lo=Const(val=0, type=int()), hi=Const(val=16, type=int()), body=[Reduce(name='Out', lhs_type=i8(), idx=[Read(name='i', idx=[], type=index()), Read(name='j', idx=[], type=index())], rhs=BinOp(op='*', lhs=Read(name='In', idx=[Read(name='i', idx=[], type=index()), Read(name='k', idx=[], type=index())], type=i8()), rhs=Read(name='Weights', idx=[Read(name='k', idx=[], type=index()), Read(name='j', idx=[], type=index())], type=i8()), type=i8()))], is_par=False)]

#### Lazy Transform Wrapper

In [13]:
from exo.stdlib.scheduling import (
    simplify,
    unroll_loop,
    reorder_loops,
    divide_loop,
    replace_all,
    
    stage_mem,
    set_memory,
    divide_dim,
    mult_dim,
    unroll_buffer,
    # join_loops,
    # cut_loop,
    # shift_loop,
    # mult_loops,
    # remove_loop
)

class ScheduledProc:
    """A wrapper class to allow for lazy evaluation of scheduling operations"""
    def __init__(self, exo_sched_func: Callable, args: list = [], kwargs: dict[str, Any] = {}):
        self.sched_func = exo_sched_func
        self.func_args = args
        self.func_kwargs = kwargs

    def _apply_transform(self, proc: Procedure) -> Procedure:
        func = self.sched_func

        return func(
            proc, 
            *self.func_args,
            **self.func_kwargs
        )

    def __repr__(self):
        return (
            f"{self.sched_func.__name__}(<proc>, "
          + ", ".join(map(str, self.func_args))
          + ", "
          + ", ".join([f"{key} = {val}" for key, val in self.func_kwargs.items()])
          + ")"
        )

    def __call__(self, proc: Procedure) -> Procedure | None:
        try:
            return self._apply_transform(proc)
        except SchedulingError as e:
            print(f"WARNING: Encountered {e}")
            return None

### Transforms

#### Valid branches logic

In [14]:
from exo.platforms.rvv import RVV

In [15]:
def get_loop_reorders(ast: exo_ast.Proc, **kwargs) -> list[ScheduledProc]:
    """Get all valid loop reordering transforms"""
    loop_pairs = get_swappable_loops(ast)
    return [
        ScheduledProc(
            reorder_loops,
            [" ".join(list(loop_pair))], 
            {}
        ) for loop_pair in loop_pairs
    ]

get_loop_reorders(PROC_AST)

[reorder_loops(<proc>, i j, ), reorder_loops(<proc>, j k, )]

In [16]:
def get_loop_unrolls(ast: exo_ast.Proc, **kwargs) -> list[ScheduledProc]:
    """Get every possible loop unrolling transform"""
    all_loops = extract_fors(ast)
    unrolls = [ ]
    for loop in all_loops:
        def unroll(proc: Procedure) -> Procedure:
            p = unroll_loop(proc, loop.name)
            return simplify(p)
        unrolls.append(ScheduledProc(unroll))

    return unrolls

get_loop_unrolls(PROC_AST)

[unroll(<proc>, , ), unroll(<proc>, , ), unroll(<proc>, , )]

In [17]:
def get_loop_splits(ast: exo_ast.Proc, **kwargs) -> list[ScheduledProc]:
    """Get every loop split given `LOOP_SPLIT_SIZES` set earlier in the notebook"""
    lowest_loops = find_innermost_loops(ast)

    splits: list[ScheduledProc] = [ ]
    for loop in lowest_loops:
        if not (isinstance(loop.hi, exo_ast.Const) and isinstance(loop.lo, exo_ast.Const)):
            continue # we can't split loops without constant bounds

        for split_const in range(LOOP_SPLIT_SIZES, loop.hi.val - loop.lo.val, LOOP_SPLIT_SIZES):
            new_loops = [f"{loop.name}_out", f"{loop.name}_in"]
            def split(proc: Procedure) -> Procedure:
                p = divide_loop(proc, loop.name, split_const, new_loops, tail="cut")
                return simplify(p)

            splits.append(ScheduledProc(split))

    return splits

get_loop_splits(PROC_AST)

[split(<proc>, , ), split(<proc>, , ), split(<proc>, , )]

In [18]:
def get_loop_fusions(ast: exo_ast.Proc, **kwargs): raise NotImplementedError
    # all_loops = extract_fors(ast)

    # fusions = [ ]
    # num_loops = len(all_loops)
    # for i in range(num_loops):
    #     for j in range(i+1, num_loops):
    #         loop1 = get_pattern_from_node(all_loops[i])
    #         loop2 = get_pattern_from_node(all_loops[j])
    #         fusions.append(ScheduledProc(join_loops, [loop1, loop2]))
    
    # return fusions
# fs = get_loop_fusions(PROC_AST)
# fs[1](get_exo_proc())

In [19]:
def get_mem_stages(proc: Procedure, **kwargs):
    """Stage every identifier outside of every block onto every memory"""
    blocks = find_all_blocks(proc.get_ast()) # pyright: ignore[reportArgumentType]
    buffers = get_names_and_dimensions(proc)

    stages = [ ]
    for buff_name, dims in buffers.items():
        for block in blocks:
            for mem in [RVV, DRAM]:

                block_pattern = get_pattern_from_node(block)
                new_symbol = f"{buff_name}Staged"
                window_expr = f"{buff_name}[{','.join([f'0:{end}' for end in dims])}]"

                def do_stage(proc):
                    proc = stage_mem(proc, block_pattern, window_expr, new_symbol)
                    proc = set_memory(proc, new_symbol, mem)
                    return simplify(proc)

                stages.append(ScheduledProc(do_stage))

    return stages
e = get_mem_stages(get_exo_proc())
len(e), e

(18,
 [do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , ),
  do_stage(<proc>, , )])

In [20]:
def get_buff_reshapes(proc: Procedure, **kwargs):
    """Get a flattening of every buffer to be of dimension (-1, 4) to eventually work with rvv"""
    buffers = get_names_and_dimensions(proc)

    reshapes = [ ]
    for buff, dims in buffers.items():
        num_flattens = len(dims) - 1
        def reshape(proc: Procedure) -> Procedure:
            proc = divide_dim(proc, f"{buff} : _", len(dims)-1, 4)
            for _ in range(num_flattens):
                proc = mult_dim(proc, buff, 0, 1)
            return simplify(proc)

        reshapes.append(ScheduledProc(reshape))

    return reshapes
get_buff_reshapes(get_exo_proc())

[reshape(<proc>, , ), reshape(<proc>, , ), reshape(<proc>, , )]

In [21]:
def get_buff_unrolls(proc: Procedure) -> list[ScheduledProc]:
    """Get an unrolling of every buffer that is of shape (-1, 4)"""
    buffers = get_names_and_dimensions(proc)

    unrolls = [ ]
    for name, dims in buffers.items():
        if len(dims) != 2 or dims[1] != 4:
            # if the buffer can't be put onto RVV, skip
            continue
        
        unrolls.append(ScheduledProc(unroll_buffer, [name, 0]))
    
    return unrolls
get_buff_unrolls(get_exo_proc())

[]

#### Gemmini instructions

This section ended up getting abandoned due to the complexity of dealing with *two* necessary memory staging transforms per output buffer. The problem was simplified to only search for RVV algorithms to improve the chance that a valid algorithm could be found in time

In [22]:
from exo.platforms.gemmini import (
    do_ld_i8_block_id1,
    do_ld_i8_vector,
    do_ld_i8,

    do_matmul_i8,

    st_acc_i32,
    do_st_acc_i8,

    config_ld_i8,
    config_ld_i8_id1,
    config_matmul,

)

from exo.API import check_eqv_proc

def find_eq_gemmini_subtrees(ast: exo_ast.QueryAST) -> list[exo_ast.QueryAST]:
    ...

def get_gemmini_insts(_: exo_ast.Proc) -> list[ScheduledProc]:
    return [
        ScheduledProc(replace_all, [gemmini_inst])
        for gemmini_inst in [
            # do_ld_i8_block_id1,
            # do_matmul_i8

        ]
    ]

ALL_INSTS = [
    "acc_scale",
] + [ # i8 loading config insts
    "config_ld_i8" + suff
    for suff in [
        "", "_id1", "_id2", "_s2_id1"
    ]
] + [ # i8 loads
    "do_ld_i8" + suff # ld_i8_prototype
    for suff in [
        "", "_id1", "_id2", "_s2_id1", 
        "_block_id1", 
        "_block_id2", 
        "_vector",
    ]
] + [ # i8 block loads (not "do")
    "ld_i8_block" + suff
    for suff in [
        "",
        "_id1", "_id1_v2", "_id1_s2_v2",
        "_id2", "_id2_v2", "_id2_s2_v2",
    ]
] + [ # remaining i8 loads (not "do")
    "ld_i8" + suff # this one is the same as do_ld_i8, but with a pass statement
    for suff in [
        "", "_v2", "_s2",
        "_id1", "_id1_v2", "_id1_s2_v2",
        "_id2", "_id2_v2", "_id2_s2_v2",
        "_vector",
    ]
] + [
    "do_ld_acc_i32",
    "do_ld_acc_i32_vector",

    "config_ld_acc_i32_vector",
    
    "ld_acc_i8",
    "ld_acc_i32",
    "ld_acc_i32_vector",
    "ld_acc_i32_vector_v2",


    "zero_block_id2",
    
    "st_i8",

    "clamp",

    "config_st_acc_i8",
    "do_st_acc_i8",

    "st_acc_i8",
    "st_acc_i8_v2",
    "st_acc_i8_s2_v2",
    
    "st_acc_i32",
    
    "config_zero",

    "do_zero_i8",
    "do_zero_i8_vector",

    "do_zero_acc_i32",

    "zero_acc_i32_v2",
    "zero_acc_i32",
    
    
    "zero_i8",
    "zero_i8_v2",
    "zero_i8_vector",
    "zero_i8_vector_v2",
    

    "config_matmul",

    "do_matmul_i8",
    # "matmul_i8_v2",
    # "matmul_i8",

    # "do_matmul_acc_i8",
    # "matmul_acc_i8_v2",
    # "matmul_acc_i8",
]

#### RVV instructions

In [23]:
import exo.platforms.rvv as r

In [24]:
def replace_rvv_ld_st(proc: Procedure) -> Procedure:
    return ( replace_all(
        replace_all(
            proc, 
            r.rvv_vld_4xf32
        )
        , r.rvv_vst_4xf32)
    )

In [25]:
def get_rvv_insts(**_) -> list[ScheduledProc]:
    return [
        ScheduledProc(replace_rvv_ld_st),
        ScheduledProc(replace_all, [r.rvv_vfmacc_1xf32_4xf32]),
        ScheduledProc(replace_all, [r.rvv_vfmacc_4xf32_4xf32]),
        ScheduledProc(replace_all, [r.rvv_vfmacc_4xf32_1xf32]),
        # ScheduledProc(replace_all, [r.rvv_broadcast_4xf32]),
        # ScheduledProc(replace_all, [r.rvv_broadcast_4xf32_0]),
        # ScheduledProc(replace_all, [r.rvv_broadcast_4xf32_scalar]),
    ]
get_rvv_insts(ast=PROC_AST)

[replace_rvv_ld_st(<proc>, , ),
 replace_all(<proc>, def rvv_vfmacc_1xf32_4xf32(dst: [f32][4] @ RVV, lhs: [f32][1] @ DRAM,
                            rhs: [f32][4] @ RVV, vl: size):
     # @instr {dst_data} = __riscv_vfmacc_vf_f32m1{dst_data}, {lhs_data}, {rhs_data},{vl});
     assert stride(dst, 0) == 1
     assert stride(lhs, 0) == 1
     assert stride(rhs, 0) == 1
     assert vl >= 0
     assert vl <= 4
     for i in seq(0, vl):
         dst[i] += lhs[0] * rhs[i], ),
 replace_all(<proc>, def rvv_vfmacc_4xf32_4xf32(dst: [f32][4] @ RVV, lhs: [f32][4] @ RVV,
                            rhs: [f32][4] @ RVV, vl: size):
     # @instr {dst_data} = __riscv_vfmacc_vv_f32m1({dst_data}, {lhs_data}, {rhs_data},{vl});
     assert stride(dst, 0) == 1
     assert stride(lhs, 0) == 1
     assert stride(rhs, 0) == 1
     assert vl >= 0
     assert vl <= 4
     for i in seq(0, vl):
         dst[i] += lhs[i] * rhs[i], ),
 replace_all(<proc>, def rvv_vfmacc_4xf32_1xf32(dst: [f32][4] @ RVV, lhs: [f32][

#### Aggregate all valid branches

In [36]:
def get_all_exo_tsfs(proc: Procedure, include_simplify: bool = False, debug: bool = False):
    """Get a list of all possible transforms that can be applied to a procedure"""
    root_ast: exo_ast.Proc = proc.get_ast() # pyright: ignore[reportAssignmentType]
    tsfs: list[ScheduledProc] = [ ]

    for expansion in [
        get_loop_reorders,
        # get_mem_stages,
        # get_buff_reshapes,
        # get_loop_unrolls,
        # get_loop_splits,
        # get_gemmini_insts,
        # get_rvv_insts,
    ]:

        new_tsfs = expansion(
            ast = root_ast,
            proc = proc,
        )

        if debug:
            print(f"\t  Found {len(new_tsfs)} cases of {expansion.__name__}")

        tsfs.extend(new_tsfs)

    if include_simplify:
        tsfs.append(ScheduledProc(simplify))

    return tsfs

p = get_exo_proc()
tsf = get_all_exo_tsfs(get_exo_proc())[0]
tsf(p)

```python
def generated_operation(In: i8[16, 16] @ DRAM, Weights: i8[16, 16] @ DRAM,
                        Out: i8[16, 16] @ DRAM):
    for j in seq(0, 16):
        for i in seq(0, 16):
            for k in seq(0, 16):
                Out[i, j] += In[i, k] * Weights[k, j]
```

### Tree-search

In [38]:
from heapq import heappop, heappush
from random import random


def apply_scheduled_proc(sp: ScheduledProc, p: Procedure):
    return sp(p), p

def estimate_cost(proc: Procedure) -> float:
    return 1.

def log_branch(parent: Procedure, child: Optional[Procedure] = None, failed: bool = False):
    parent_hash = hash(extract_c_code(parent))
    if child is None:
        print("#", parent_hash, "revisited")
        return
    child_hash = hash(extract_c_code(child))
    print(parent_hash, "->", child_hash, "# failed" if failed else "")

evaluation_queue = PersistentQueue(persistence_file=QUEUE_FILE, overwrite=True)
@profileit(do_print=True)
def search_procs(proc: Procedure, debug: bool = False, compile_to_c: bool = True):
    """Perform a randomized breadth-first search of all possible sequences of transformations of the procedure"""
    # og_c_code = str(proc) if not compile_to_c else extract_c_code(proc)
    # evaluation_queue.put(og_c_code)
    seen_procs = {
        # og_c_code : proc
    }

    visitation_heap = [ ]
    heappush(visitation_heap, (0, 0, proc))

    while len(visitation_heap) > 0:
        _, _, p = heappop(visitation_heap)
        code = str(p) if not compile_to_c else extract_c_code(p)

        if code in seen_procs: # this is a previously visited node, skip expansion
            log_branch(p, None)
            continue

        seen_procs[code] = p
        evaluation_queue.put(code)

        all_tsfs = get_all_exo_tsfs(p)

        new_procs = [
            (tsf(p), tsf)
            for tsf in 
            all_tsfs
        ]

        for np, tsf in new_procs:
            log_branch(p, np)
            if np is None:
                continue # filter out failed transforms

            heappush(visitation_heap, (
                estimate_cost(np), random(), np
            )) #                     ^
               # We break ties with randomness

    return list(seen_procs.values())


    # def deeper(p: Procedure, can_split: bool = False, debug_prefix: str = ""):
    #     nonlocal seen_procs

    #     all_tsfs = get_all_exo_tsfs(p)

    #     if can_split and len(all_tsfs) > 1:
    #         print("splitting")
    #         with concurrent.futures.ProcessPoolExecutor() as executor:
    #             futures = [
    #                 executor.submit(apply_scheduled_proc, t, p)
    #                 for t in all_tsfs
    #             ]
    #             new_procs = [
    #                 f.result() # the un-pickle-ing of the return values doesn't work:
    #                            #    - When pickling, the symbols get packed in as they 
    #                            #        understand themselves (i.e. <thing>.__getstate__)
    #                            #    - The problem is that old objects don't expect to be a 
    #                            #        part of a more complex construct and so return a
    #                            #        __main__.<thing> value for that
    #                            #    - However, for us, it's often more complicated -- including
    #                            #        imports and submodules, e.g. exo.<thing> due to exo 
    #                            #        importing <thing> all the way up to its root __init__
    #                            #    - So when pickle tries to write to what <thing> thinks it is
    #                            #        and finds that that name doesn't actually exist in our
    #                            #        namespace, it errors
    #                 for f in futures
    #             ]
    #             # new_procs = list( # this is a much shorter version of the above (if it worked lol)
    #             #     executor.map(apply_scheduled_proc, [[t, p] for t in all_tsfs])
    #             # )
    #     else:
    #         new_procs = [ # currently, this calculation is the performance bottleneck, as this is entirely serial
    #             (tsf(p), tsf) for tsf in 
    #             all_tsfs
    #         ]
    #     if debug: print(f"{debug_prefix}Found {len(new_procs)} potential procs")
    #     actually_new_procs = []

    #     # here is where we would add the secret sauce:
    #     #new_procs = sort_by_fancy_heuristic(new_procs)
        
    #     for new_proc, tsf in new_procs:
    #         if new_proc is None: # don't compile to C if we already know its a dupe
    #             continue

    #         c_code = str(new_proc) if not compile_to_c else extract_c_code(new_proc)
        
    #         if c_code in seen_procs.keys():
    #             continue

    #         seen_procs[c_code] = new_proc
    #         evaluation_queue.put(c_code)
    #         actually_new_procs.append((new_proc, tsf))
        
    #     if debug: print(f"{debug_prefix}branching into {len(actually_new_procs)} new procs")

    #     for new_proc, tsf in actually_new_procs:
    #         deeper(
    #             new_proc,
    #             debug_prefix = debug_prefix+"\t"
    #         )
    
    # deeper(proc)

    # return list(seen_procs.values())

procs = search_procs(get_exo_proc())
procs, len(procs)

         2144076 function calls (2128027 primitive calls) in 1.073 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.086    1.086 2961857648.py:20(search_procs)
      6/1    0.000    0.000    1.062    1.062 2961857648.py:64(deeper)
        6    0.000    0.000    0.972    0.162 2961857648.py:95(<listcomp>)
       12    0.000    0.000    0.972    0.081 1144370832.py:45(__call__)
       12    0.000    0.000    0.972    0.081 1144370832.py:27(_apply_transform)
       12    0.000    0.000    0.972    0.081 API_scheduling.py:79(__call__)
       12    0.000    0.000    0.572    0.048 API_scheduling.py:57(__call__)
       12    0.000    0.000    0.572    0.048 API_scheduling.py:589(_cursor_call)
       12    0.000    0.000    0.569    0.047 API_scheduling.py:406(_cursor_call)
       12    0.000    0.000    0.569    0.047 API.py:270(find)
       12    0.000    0.000    0.569    0.047 API_cursors.py:9

([<exo.API.Procedure at 0x7fffa4c1b5b0>,
 6)

# Exo's scheduling API


## Top-level Python function decorator

1. `@proc` - decorates a Python function which is parsed and compiled as Exo. Replaces
   the function with a `Procedure` object.
2. `@instr` - same as `@proc`, but accepts a hardware instruction as a format string.
3. `@config` - decorates a Python class which is parsed and compiled as an Exo
   configuration object



## Procedure object methods



**Introspection operations**

- `.name()` returns the procedure name.
- `.check_effects()` forces Exo to run effect checking on the procedure.
- `.show_effects()` prints the effects of the procedure.
- `.show_effect(stmt)` prints the effect of the `stmt` in the procedure.
- `.is_instr()` returns `true` if the procedure has a hardware instruction string.
- `.get_instr()` returns the hardware instruction string.
- `.get_ast()` returns a `QAST`, which is an AST representation suitable for
  introspection.



**Execution / interpretation operations**

- `.compile_c(directory, filename)` compiles the procedure into C and stores
  in `filename` in the `directory`.
- `.interpret(**args)` runs Exo interpreter on the procedure.



## Scheduling operations on Procedure objects



### Loop related operations



- `.split(loop, split_const, iter_vars, tail='guard', perfect=False)`
  - Splits `loop` into an outer and an inner loop. The inner loop bound is `split_const` and the outer and inner loop names are specified by a list of strings `iter_vars`. If `perfect` is True, it will not introduce a tail case. `tail` specifies the tail strategies, where the options are `guard`, `cut`, and `cut_and_guard`.
- `.fuse_loop(loop1, loop2)`
  - Fuses two adjacent loops with a common iteration variable.
- `.partition_loop(loop, num)`
  - Partitions `loop` into two loops, the first running between `0` and `num` and the second between `num+1` and `loop`'s original bound.
- `.reorder(loop1, loop2)`
  - Reorders two nested loops. `loop2` should be nested directly inside `loop1`. `loop1` will be nested inside `loop2` after this operation.
- `.unroll(loop)`
  - Unrolls the loop. The loop needs to have a constant bound.
- `.fission_after(stmt, n_lifts=1)`
  - Fissions the `n_lifts` number of loops around the `stmt`. The fissioned loops around the `stmt` need to be directly nested with each other and the statements before and after the `stmt` should not have any allocation dependencies.
- `.remove_loop(loop)`
  - Replaces the loop with its body if the body is idempotent. The system must be able to prove that the loop runs at least once.



### Buffer related operations



- `.reuse_buffer(buf1, buf2` 
  - Reuses a buffer `buf1` in the use site of `buf2` and removes the allocation of `buf2`
- `.inline_window(win_stmt)`
  - Removes the window statement `win_stmt`, which is an alias to the window, and inlines the windowing in its use site
- `.expand_dim(stmt, alloc_dim, indexing)`
  - Expands the dimension of the allocation statement `stmt` with dimension `alloc_dim` of indexing `indexing`
- `.bind_expr(new_name, expr)`
  - Binds the right hand side expression `expr` to a newly allocated buffer named `new_name`
- `.stage_mem(win_expr, new_name, stmt_start, stmt_end=None)`
  - Stages the buffer `win_expr` to the new window expression `new_name` in statement block (`stmt_start` to `stmt_end`), and adds an initialization loop and a write-back loop
- `.rearrange_dim(alloc, dimensions)`
  - Takes an allocation statement and a list of integers to map the dimension. It rearranges the dimensions of `alloc` in `dimension` order. E.g., if `alloc` were `foo[N,M,K]` and the `dimension` were `[2,0,1]`, it would become `foo[K,N,M]` after this operation.
- `.lift_alloc(alloc, n_lifts=1, keep_dims=False)`
  - Lifts the allocation statement `alloc` out of `n_lifts` number of scopes. If and For statements are the only statements in Exo which introduce a scope. When lifting the allocation out of a for loop, it will expand its dimension to the loop bound if `keep_dims` is True.



### Config related operations



- `.bind_config(expr, config, field)`
  - Binds the right hand side `expr` to `config.field`. It will replace the use site of `expr` with `config.field` and introduces a config statement of `config.field = expr`.
- `.configwrite_root(config, field, expr)`
  - Inserts the config statement `config.field = expr` in the beginning of the procedure.
- `.configwrite_after(stmt, config, field, expr)`
  - Inserts the config statement `config.field = expr` after `stmt`.
- `.delete_config(stmt)`
  - Deletes the configuration statement.



### Other scheduling operations



- `.add_assertion(assertion)`
  - Asserts the truth of the expression `assertion` at the beginning of the procedure.
- `.lift_if(if, n_lifts=1)`
  - Lifts the if statement `if` out of `n_lifts` number of scopes. This is similar to `reorder()`, but for if statements.
- `.eliminate_dead_code(stmt)`
  - Eliminates `if` statement if condition is always True or False. Eliminates `for` statement if condition is always False.
- `.delete_pass()`
  - Deletes a `Pass` statement in the procedure.
- `.reorder_stmts(stmt1, stmt2)`
  - Reorder two adjacent statements `stmt1` and `stmt2`. After this operation, the order will be `stmt2` `stmt1`.
  - `.reorder_before(stmt)`
    - Move the statement `stmt` before the previous statement. This is a shorthand for `reorder_stmts()`.
- `.replace(subproc, stmt)`
  - Replace the statement with a call to `subproc`. This operation is one of our contributions and is explained in detail in the paper.
- `.replace_all(subproc)`
  - Eagerly replace every matching statement with a call to `subproc`.
- `.inline(call_site)`
  - Inline the function call.
- `.is_eq(another_proc)`
  - Returns True if `another_proc` is equivalent to the procedure.
- `.call_eqv(eqv_proc, call_site)`
  - Replace the function call statement of `call_site` with a call to an equivalent procedure `eqv_proc`.
- `.repeat(directive, *args)`
  - Continue to run the directive until it fails. The directive and its arguments are given separately, e.g. `proc.repeat(Procedure.inline, "proc_to_inline(_)")`
- `.simplify()`
  - Simplify the code in the procedure body. Tries to reduce expressions to constants and eliminate dead branches and loops. Uses branch conditions to simplify expressions inside the branches.
- `.rename(new_name)`
  - Rename this procedure to `new_name`.
- `.make_instr(instr_string)`
  - Converts this procedure to an instruction procedure with instruction `instr_string`.
- `.partial_eval(*args, **kwargs)`
  - Specializes this procedure to the given argument values.
- `.set_precision(name, type)`
  - Sets the precision type of `name` to `type`.
- `.set_window(name, is_window)`
  - If `is_window` is True, it sets the buffer `name` to window type, instead of a tensor type.
- `.set_memory(name, mem_type)`
  - Sets a buffer `name`'s memory type to `mem_type`.
